In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm

from scipy.stats import linregress
from scipy.stats import mannwhitneyu
from scipy.stats import ks_2samp
from scipy.stats import rankdata

import matplotlib as mpl
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
PROTEINS = [
    "5P",
    "NSP1",
    "NSP2",
    "NSP3",
    "NSP4",
    "NSP5",
    "NSP6",
    "NSP7",
    "NSP8",
    "NSP9",
    "NSP10",
    "NSP11",
    "NSP12",
    "NSP13",
    "NSP14",
    "NSP15",
    "NSP16",
    "Spike",
    "NS3",
    "E",
    "M",
    "NS6",
    "NS7a",
    "NS7b",
    "NS8",
    "N",
    "NS9b",
    "NS9c",
    "3P",
    "RNA"
]

ANNOTATION = {
    "5P": "1-265",
    "NSP1": "266-805",
    "NSP2": "806-2719",
    "NSP3": "2720-8554",
    "NSP4": "8555-10054",
    "NSP5": "10055-10972",
    "NSP6": "10973-11842",
    "NSP7": "11843-12091",
    "NSP8": "12092-12685",
    "NSP9": "12686-13024",
    "NSP10": "13025-13441",
    "NSP11": "13442-13480",
    # "NSP12": "13442-13468",
    "NSP12": "13468-16236",
    "NSP13": "16237-18039",
    "NSP14": "18040-19620",
    "NSP15": "19621-20658",
    "NSP16": "20659-21552",
    "Spike": "21563-25384",
    "NS3": "25393-26220",
    "E": "26245-26472",
    "M": "26523-27191",
    "NS6": "27202-27387",
    "NS7a": "27394-27759",
    "NS7b": "27756-27887",
    "NS8": "27894-28259",
    "N": "28274-29533",
    "NS9b": "28284-28577",
    "NS9c": "28734-28955",
    "3P": "29534-30331"
}

In [3]:
def read_fasta(path):
    result = ""
    description = ""

    ff = open(path, "r")

    line = next(ff, None)
    while line:
        line = line.rstrip("\n")
        if not line:
            line = next(ff, None)
            continue

        if line.startswith(">"):
            if result:
                yield description, result
            description = line.lstrip(">")
            result = ""
        else:
            result += line

        line = next(ff, None)

    yield description, result
    ff.close()
    
def percent(seq, chr="N"):
    count = 0
    for char in seq:
        if char == chr:
            count += 1
    return count / len(seq)

def region_count(reg, start=None, end=None):
    reg = reg.lstrip("[").rstrip("]").split(", ")
    if not (start is None) and not (end is None):
        reg = [s for s in reg if s and (start <= int(s) <= end)]
    else:
        reg = [s for s in reg if s]
    return len(reg)

In [4]:
meta_df = pd.read_csv(
    "../../gisaid/germany_ber_meta.csv",
    sep=",", index_col=0)

rna_df = pd.DataFrame(columns=["strain", "length", "polyA", "percN"])
tqdrator = tqdm.tqdm(read_fasta("../../gisaid/germany_ber.fasta"), total=24778)
for covid, rna in tqdrator:
    length = len(rna)
    perc = percent(rna, "N")
    poly = len(rna) - len(rna.rstrip("A"))
    append_df = pd.DataFrame(
        columns=rna_df.columns.to_list(),
        data=[[
            covid,
            length,
            poly,
            perc
        ]]
    )
    rna_df = pd.concat([rna_df, append_df], ignore_index=True)

rna_df.index = rna_df["strain"]
rna_df = rna_df.drop(columns=["strain"])
meta_df = meta_df.join(rna_df, how="inner")

meta_df["date"] = meta_df.index.str.split("|").str[1]
meta_df["date"] = pd.to_datetime(meta_df["date"], format="%Y-%m-%d")
meta_df = meta_df.sort_values(by=["date"])

meta_df["date"] = meta_df.index.str.split("|").str[1]
meta_df["date"] = pd.to_datetime(meta_df["date"], format="%Y-%m-%d")
meta_df = meta_df.sort_values(by=["date"])

meta_df = meta_df.loc[meta_df["date"].astype("str").apply(lambda dt: len(dt.split("-"))) == 3]
meta_df["year"] = meta_df["date"].astype("str").str.split("-").str[0].astype("int")
meta_df["month"] = meta_df["date"].astype("str").str.split("-").str[1].astype("int")
meta_df["day"] = meta_df["date"].astype("str").str.split("-").str[2].astype("int")
meta_df["days"] = meta_df["day"] + meta_df["month"] * 30 + meta_df["year"] * 365
start = int(meta_df.iloc[0]["days"])
meta_df["days"] -= start
meta_df = meta_df.drop(columns=["year", "month", "day"])

100%|██████████| 24778/24778 [01:57<00:00, 211.12it/s]


In [5]:
df = meta_df.loc[
    (~meta_df["scorpio_call"].isna()) & \
    (meta_df["percN"] == 0.) & \
    (meta_df["length"] > 29000)
]

STRAINS = []
df["strain"] = df.index
STRAINS = df["strain"].to_list()

<ipython-input-5-16c02205110c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["strain"] = df.index


In [11]:
meta_df = meta_df.loc[STRAINS]
meta_df.loc[:, "group"] = "first"
meta_df.loc[meta_df["scorpio_call"].str.contains("micron"), "group"] = "second"

In [7]:
# binding regions

In [8]:
rdf = pd.read_csv(f"../../results/germany_ber/seed.csv", sep=";", index_col=0)
rdf.index = rdf.index.str.lstrip(">")
rdf["strain"] = rdf.index

In [62]:
PATH_TO_DB = "/home/dude/huge/bulk/TCGA/TCGA-LUAD/miRNA/miRNA_counts.tsv"
PATH_TO_CPM = "/home/dude/huge/bulk/TCGA/TCGA-LUAD/miRNA/miRNA_CPM.tsv"
PATH_TO_EXP = "/home/dude/huge/dude/long-covid/miRNA/expressed_LUAD.csv"
PATH_TO_PT = "/home/dude/huge/bulk/TCGA/TCGA-LUAD/miRNA/tumor_normal_samples.tsv"
# PATH_TO_DB = "/home/dude/huge/bulk/TCGA/TCGA-COAD/miRNA/miRNA_counts.tsv"
# PATH_TO_CPM = "/home/dude/huge/bulk/TCGA/TCGA-COAD/miRNA/miRNA_CPM.tsv"
# PATH_TO_PT = "/home/dude/huge/bulk/TCGA/TCGA-COAD/miRNA/tumor_normal_samples.tsv"
# PATH_TO_EXP = "/home/dude/huge/dude/long-covid/miRNA/expressed_COAD.csv"
PATH_TO_MIRBASE = "/home/dude/huge/bulk/miRBase/miRBase_22.1.tsv"

def random_miRNAs(count):
    db = pd.read_csv(PATH_TO_DB, sep="\t", index_col=0)
    cpm_db = pd.read_csv(PATH_TO_CPM, sep="\t", index_col=0)
    exp_db = pd.read_csv(PATH_TO_EXP, sep=",")
    pt = pd.read_csv(PATH_TO_PT, sep="\t")

    db = db[pt["Normal ID"].to_list()]
    db["sum"] = db.sum(axis=1)
    db = db.sort_values(by=["sum"], ascending=False)
    db["cumsum"] = db["sum"].cumsum(axis=0)
    db["cumsum"] = db["cumsum"] / db["sum"].sum()

    for ptid in pt["Normal ID"].to_list():
        cpm_db[ptid] = cpm_db[ptid].apply(lambda x: 2**x)
    cpm_db["mean"] = cpm_db[pt["Normal ID"].to_list()].mean(axis=1)
    db = db[["cumsum"]].join(cpm_db[["mean"]], how="inner")

    db = db.drop(columns=["cumsum"])

    # extract MIMATS
    miRBase = pd.read_csv(PATH_TO_MIRBASE, sep="\t")
    miRBase = miRBase[["miRNA", "MIMAT"]]

    miRBase.index = miRBase["miRNA"]
    miRBase = miRBase.join(db, how="inner").drop_duplicates()
    miRBase = miRBase.rename(columns={"mean" : "CPM"})
    
    index = len(exp_db)
    for i in range(count):
        result_db = miRBase.iloc[
            index + np.random.permutation(
                len(miRBase) - index
            )[:index]
        ]
        
        result_db.loc[:, "CPM"] = exp_db["CPM"].to_list()
        
        yield result_db

In [63]:
# copy of binding regions df

In [32]:
mrdf = rdf.loc[STRAINS]

In [64]:
distribution = []

for miRNA_df in tqdm.tqdm(random_miRNAs(1000), total=1000):
    miRNA_df.index = miRNA_df["MIMAT"]

    TOTAL_EXPRESSION = miRNA_df["CPM"].sum()
    MIMATS = miRNA_df["MIMAT"].to_list()
    
    count_df = mrdf[MIMATS]  
    count_df.loc[:, "count"] = 0
    count_df.loc[:, "weighted count"] = 0
        
    s, e = None, None
    for mimat in MIMATS:
        count_df[mimat] = count_df[mimat].apply(lambda x: region_count(x, s, e))
        expression = (miRNA_df.loc[mimat, "CPM"] / TOTAL_EXPRESSION)
        count_df["count"] += count_df[mimat]
        count_df["weighted count"] += count_df[mimat] * expression
    
    count_df = count_df[["count", "weighted count"]].join(
        meta_df["group"], how="left"
    )
    
    try:
        wstat, wpv = mannwhitneyu(
            count_df.loc[count_df["group"] == "first"]["weighted count"],
            count_df.loc[count_df["group"] == "second"]["weighted count"]
        )
    except:
        wstat, wpv = 0, 1
    fwm = count_df.loc[count_df["group"] == "first"]["weighted count"].mean()
    swm = count_df.loc[count_df["group"] == "second"]["weighted count"].mean()
    
    try:
        stat, pv = mannwhitneyu(
            count_df.loc[count_df["group"] == "first"]["count"],
            count_df.loc[count_df["group"] == "second"]["count"]
        )
    except:
        stat, pv = 0, 1
    
    fm = count_df.loc[count_df["group"] == "first"]["count"].mean()
    sm = count_df.loc[count_df["group"] == "second"]["count"].mean()
    
    distribution.append([fwm, swm, wstat, wpv, fm, sm, stat, pv])

  0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
<ipython-input-64-2cf23e2e750c>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [65]:
distribution = pd.DataFrame(data=distribuiton, columns=None)

In [66]:
distribution.head()

,0,1,2,3,4,5,6,7
0,12.438418,12.650240,146473.0,0.000000e+00,414.627404,414.390942,1510781.5,2.403683e-05
1,11.140876,11.149957,1457936.0,2.642649e-02,384.106370,383.680572,1495919.5,2.892446e-04
2,12.516915,12.457914,2075397.0,4.918278e-132,460.768630,463.024434,501579.0,7.032675e-233
3,13.397601,13.300172,1933565.0,6.474194e-83,445.510216,443.430870,2318351.5,2.533695e-248
4,11.046047,10.864697,2774681.0,0.000000e+00,369.620192,370.490465,1074089.0,1.380914e-31


In [53]:
len(distribution.loc[
    (distribution[2] > 2453380) & \
    (distribution[0] > distribution[1])
]) / len(distribution)

0.13

In [61]:
len(distribution.loc[
    # (distribution[2] > 2453380) & \
    # (distribution[0] - distribution[1] > 7.44 - 7.26) &
    (distribution[4] - distribution[5] > 386.89 - 381.26)
]) / len(distribution)

0.03

In [72]:
len(distribution.loc[
    # (distribution[2] > 2453380) & \
    (distribution[0] - distribution[1] > 7.24 - 7.11) &
    (distribution[4] - distribution[5] > 315.36 - 312.77)
]) / len(distribution)

0.07

In [91]:
coad_df = pd.read_csv("/home/dude/huge/dude/long-covid/miRNA/expressed_COAD.csv", sep=",")
luad_df = pd.read_csv("/home/dude/huge/dude/long-covid/miRNA/expressed_LUAD.csv", sep=",")

In [92]:
print(len(set(luad_df["MIMAT"]) & set(coad_df["MIMAT"])))
print(len(set(luad_df["MIMAT"])))
print(len(set(coad_df["MIMAT"])))

21
32
40


In [87]:
coad_df

,miRNA,MIMAT,CPM
0,hsa-let-7a-5p,MIMAT0000062,67548.420342
1,hsa-let-7b-5p,MIMAT0000063,135685.253863
2,hsa-let-7c-5p,MIMAT0000064,4047.679411
3,hsa-let-7d-3p,MIMAT0004484,9162.228080
4,hsa-miR-100-5p,MIMAT0000098,2391.640581
5,hsa-miR-103a-3p,MIMAT0000101,11728.474926
6,hsa-miR-10a-5p,MIMAT0000253,13259.412357
7,hsa-miR-10b-5p,MIMAT0000254,13159.013451
8,hsa-miR-125a-5p,MIMAT0000443,6241.084463
9,hsa-miR-125b-5p,MIMAT0000423,2302.887904


In [94]:
luad_df.index = luad_df["miRNA"]
coad_df.index = coad_df["miRNA"]
result_df = luad_df[["CPM"]].join(
    coad_df[["CPM"]],
    lsuffix="LUAD",
    rsuffix="COAD",
    how="inner"
)

result_df["factor"] = result_df["CPMCOAD"] / result_df["CPMLUAD"]
result_df.sort_values(by=["factor", "CPMCOAD", "CPMLUAD"], ascending=False)

,CPMLUAD,CPMCOAD,factor
miRNA,,,
hsa-miR-92a-3p,15279.448538,119494.399112,7.820596
hsa-miR-200c-3p,9697.790179,59543.779003,6.139933
hsa-let-7b-5p,50573.293235,135685.253863,2.682943
hsa-miR-99b-5p,61016.821090,157162.438521,2.575723
hsa-miR-10b-5p,11459.885223,13159.013451,1.148267
hsa-let-7c-5p,5322.550998,4047.679411,0.760477
hsa-miR-25-3p,8765.517727,6031.839503,0.688133
hsa-let-7a-5p,111185.122439,67548.420342,0.607531
hsa-miR-23a-3p,4072.305528,2467.000225,0.605799
